In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft, fft2, ifft2
from scipy import signal

Что такое дискретная свертка?
Есть два сигнала $a[n], n \in [0, N)$ и $b[m], m \in [0, M)$. Тогда третий сигнал $c[k], k \in [0, N+M-1)$ называется сверткой, если:
$$ c[k] = \sum\limits_{j=-\infty}^{\infty} a[j]b[k-j] = \sum\limits_{j=-\infty}^{\infty} a[k-j]b[j] .$$

Предположим, что $M < N$ и возьмем второе равенство и избавимся от бесконечностей, т.к. мы работаем с конечными сигналами.
$$ c[k] = \sum\limits_{j=j_1}^{j_2} a[k-j]b[j] ,$$

Осталось определить $j_1,\ j_2$. При $m \notin [0, M) \ b[m] = 0$. Тогда $j_1 >= 0$, $j_2 < M$

Пойдем дальше. Так как при $n \notin [0, N) \ a[n] = 0$, то $k-j_2 \ge 0$ и $k-j_1 < N$. 

Из этого следует, что $j_1 = max(k-N+1, 0)$, а $j_2 = min(k,M-1)$. Так как в программе мы будем использовать $j_2$ в качестве верхней границы полуинтервала, то можно переписать в виде $j_2 = min(k+1,M)$.

Можно показать почему $k \in [0, N+M-1)$. Введем обозначение $k \in [0, k_{max})$. $k_{max}$ - это такое значение $k$ при котором $j_1 = j_2$, т.е. полуинтервал индексов пуст. Запишем
$$ max(k_{max}-N+1,0) = min(k_{max}+1,M) .$$

Так как мы знаем, что свертка длиннее исходных последовательностей, получаем равенство:
$$ k_{max}-N+1 = M$$
$$ k_{max} = N + M - 1$$

In [2]:
def conv(a, b):
    c = [0] * (len(a) + len(b) - 1)

    for k in xrange(len(c)):
        j_start = max(k - len(a) + 1, 0)
        j_end = min(k + 1, len(b))
    #     print "i:", i, "j_start:", j_start, "j_end:", j_end
        for j in xrange(j_start, j_end):
            c[k] += a[k - j] * b[j]
    return c

In [3]:
a = np.array([1, 2, 0, 3, 2, 5, 1, 0, 2])
b = np.array([3, 4, 0, 0, 1])
print conv(a, b)

[3, 10, 8, 9, 19, 25, 23, 7, 8, 13, 1, 0, 2]


Аналогично для двумерной свертки

In [4]:
def conv2(a, b):
    N = a.shape[0] + b.shape[0] - 1
    M = a.shape[1] + b.shape[1] - 1
    c = np.zeros((N, M))
    
    for n in xrange(N):
        for m in xrange(M):
            i_start = max(n - a.shape[0] + 1, 0)
            i_end = min(n + 1, b.shape[0])
            
            j_start = max(m - a.shape[1] + 1, 0)
            j_end = min(m + 1, b.shape[1])
            
            for i in xrange(i_start, i_end):
                for j in xrange(j_start, j_end):
                    c[n, m] += a[n-i, m-j] * b[i, j]
    return c

In [5]:
a = np.array([[1, 2, 0, 3, 7], 
              [0, 10, 1, 1, 1], 
              [32, 12, 0, 0, 0],
              [2, 1, 3, 5, 0],
              [3, 0, 4, 2, 1]])

b = np.array([[1, 0, 1],
              [2, 1, 0],
              [0, 1, 1]])

print conv2(a,b)

[[  1.   2.   1.   5.   7.   3.   7.]
 [  2.  15.   3.  17.  19.   8.   1.]
 [ 32.  33.  47.  17.   6.  11.   7.]
 [ 66.  57.  27.  17.   5.   7.   1.]
 [  7.  36.  58.  27.  10.   2.   1.]
 [  6.   5.  11.  12.  12.   6.   0.]
 [  0.   3.   3.   4.   6.   3.   1.]]


Говорят, что для больших последовательностей выгоднее вычислять свертку используя теорему о свертке.
$$ F(a*b) = F(a)F(b) $$

In [6]:
def fourier_conv(a, b):
    Nout = len(a) + len(b) - 1

    a = np.pad(a, (0, Nout-len(a)), "constant", constant_values=0)
    b = np.pad(b, (0, Nout-len(b)), "constant", constant_values=0)
    
    ab = ifft(fft(a) * fft(b))

    ab = np.real_if_close(ab)
    ab[abs(ab) < 1e-12] = 0.0
    return ab

In [7]:
a = np.array([1, 2, 0, 3, 2, 5, 1, 0, 2])
b = np.array([3, 4, 0, 0, 1])
print fourier_conv(a, b)
print conv(a, b)

[  3.  10.   8.   9.  19.  25.  23.   7.   8.  13.   1.   0.   2.]
[3, 10, 8, 9, 19, 25, 23, 7, 8, 13, 1, 0, 2]


Теперь попробуем вычислить так называемую обратную свертку. Пусть $c = a * b$ и нам известны $b$ и $c$, а мы хотим найти $a$.
$$ F(c) = F(a*b) = F(a)F(b) $$
$$ F(a) = F(c) / F(b) $$
$$ a = F^{-1}(F(a)) = F^{-1}(F(c) / F(b)) $$

In [8]:
def deconv(c, b):
    Nout = len(c) - len(b) + 1
    b = np.pad(b, (0, len(c)-len(b)), "constant", constant_values=0)

    a = ifft(fft(c) / fft(b))
    a = np.real_if_close(a[:Nout])
    a[abs(a) < 1e-12] = 0.0
    return a

In [9]:
a = np.array([1, 2, 0, 3, 2, 5, 1, 0, 2])
b = np.array([3, 4, 0, 0, 1])
c = signal.convolve(a, b)

print a
print deconv(c, b)

[1 2 0 3 2 5 1 0 2]
[ 1.  2.  0.  3.  2.  5.  1.  0.  2.]


In [10]:
def deconv2(c, b):
    Nout = c.shape[0] - b.shape[0] + 1
    Mout = c.shape[1] - b.shape[1] + 1
    b = np.pad(b, [(0, c.shape[0] - b.shape[0]), (0, c.shape[1] - b.shape[1])], "constant", constant_values=0)
    
    a = ifft2(fft2(c) / fft2(b))
    a = np.real_if_close(a[:Nout, :Mout])
    a[abs(a) < 1e-12] = 0.0
    return a

In [11]:
a = np.array([[1, 2, 0, 3, 7], 
              [0, 10, 1, 1, 1], 
              [32, 12, 0, 0, 0],
              [2, 1, 3, 5, 0],
              [3, 0, 4, 2, 1]])

b = np.array([[1, 0, 1],
              [2, 1, 0],
              [0, 1, 1]])

c = signal.convolve2d(a, b)

print a
print deconv2(c, b)

[[ 1  2  0  3  7]
 [ 0 10  1  1  1]
 [32 12  0  0  0]
 [ 2  1  3  5  0]
 [ 3  0  4  2  1]]
[[  1.   2.   0.   3.   7.]
 [  0.  10.   1.   1.   1.]
 [ 32.  12.   0.   0.   0.]
 [  2.   1.   3.   5.   0.]
 [  3.   0.   4.   2.   1.]]
